In [1]:
from torch.utils.data import DataLoader

from data.dataset_class import MultiLabelDataset
from cnn.base_model import ResNetModel
from cnn.run_train import run_epochs, test_model


d:\pr\cnn_with_resnet\.venv\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [2]:
model = ResNetModel()

In [4]:
train_data = DataLoader(MultiLabelDataset("train"), batch_size=64, shuffle=True)
val_data = DataLoader(MultiLabelDataset("val"), batch_size=64, shuffle=False)
test_data = DataLoader(MultiLabelDataset("test"), batch_size=64, shuffle=False)

In [5]:
history = run_epochs(1, train_data, val_data, model)

  0%|          | 0/283 [00:00<?, ?it/s]

Epoch: [1/1], train_loos: 0.6925, train_acc: 0.9206, val_loos: 0.4981,  val_acc: 0.9187


In [6]:
print(test_model(test_data, model))

Loss: 0.5566639097531637
Accuracy: 0.9207202505219206


In [ ]:
example_data = MultiLabelDataset("test")

img, target = example_data[123]
pred = model.predict_proba(img.unsqueeze(0))

print("Target: ", target)
print("Predict", [round(float(x), 2) for x in pred.numpy()[0]])

<class 'torchvision.tv_tensors._image.Image'>
Target:  tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.])
Predict [0.0, 0.0, 0.0, 0.0, 0.03, 0.0, 0.0, 0.0, 1.0, 0.02, 0.0, 0.02, 0.36, 0.0, 0.0, 0.0]


In [22]:
print(model.predict(img.unsqueeze(0)))

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])


In [7]:

print(test_model(test_data, model))

Loss: 1.1013838120301565
Accuracy: 0.9111038622129436


In [6]:
model = ResNetModel.load_checkpoint(0)



Metrics this checkpoint: {'train_loos': 0.7193420712602433, 'train_acc': 0.9324925406122223, 'val_loos': 1.0422675653050344, 'val_acc': 0.9047988881621976}


In [ ]:
print(test_model(test_data, model))